In [1]:
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.initializers import *
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.regularizers import l1, l2

Using TensorFlow backend.


In [2]:
class Data:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.input_shape = (112, 112, 3)

    def loadTrainData(self, trainFile, trainLabelsFile):
        trainData = np.load(trainFile)
        trainData = trainData.reshape(trainData.shape[0], trainData.shape[2], trainData.shape[3], trainData.shape[1])

        trainLabels = np.load(trainLabelsFile)
        trainLabels = keras.utils.to_categorical(trainLabels)

        trainData, trainLabels = shuffle(trainData, trainLabels)
        
        validationSize = 12000
        validationData = trainData[validationSize:]
        validationLabels = trainLabels[validationSize:]

        trainData = trainData[:validationSize]
        trainLabels = trainLabels[:validationSize]

        return trainData, trainLabels, validationData, validationLabels

    def loadTestData(self, testFile, testLabelsFile):
        testData = np.load(testFile)
        testData = testData.reshape(testData.shape[0], testData.shape[2], testData.shape[3], testData.shape[1])

        testLabels = np.load(testLabelsFile)
        testLabels = keras.utils.to_categorical(testLabels)

        return testData, testLabels



In [3]:
class Model:
    def __init__(self, input_dim, classes, lr=0.001, epochs=1):
        self.input_dim = input_dim
        self.num_classes = classes
        self.learning_rate = lr
        self.epochs = epochs

    def lr_schedule(self, epoch):
        if epoch % 10 == 0:
            self.learning_rate = self.learning_rate/10
        print('Learning rate: ', self.learning_rate)
        return self.learning_rate

    def model(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal', activation='relu', input_shape=self.input_dim))
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
        model.add(Conv2D(64, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
        model.add(BatchNormalization(momentum=0.9, epsilon=0.0001))
        
        model.add(AveragePooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.4))

        model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='relu', kernel_initializer='he_normal'))
        model.add(Conv2D(512, kernel_size=(5, 5), padding='same', activation='relu', kernel_initializer='he_normal'))
        model.add(BatchNormalization(momentum=0.9, epsilon=0.0001))
        
        model.add(GlobalAveragePooling2D())
        model.add(Dropout(0.4))
        
        model.add(Dense(512, kernel_initializer='he_normal', activation='relu'))   
        model.add(Dense(128, kernel_initializer='he_normal', activation='relu'))            
        model.add(Dense(64, kernel_initializer='he_normal', activation='relu'))
        model.add(Dense(32, kernel_initializer='he_normal', activation='relu'))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer=Adam(lr=self.lr_schedule(1)), #lr=self.learning_rate, decay=self.learning_rate/self.epochs),
                      loss=keras.losses.categorical_crossentropy,
                      metrics=['accuracy'])
        return model



In [4]:
trainDataFile = "/kaggle/input/minidata/train.npy"  # sys.argv[1]
trainLabelsFile = "/kaggle/input/minidata/trainlabels.npy"  # sys.argv[2]
modelFile = "model64.h5"  # sys.argv[3]

BATCH_SIZE = 32
LEARNING_RATE = 1e-2
EPOCHS = 50
NUM_CLASSES = 10

dataObj = Data(BATCH_SIZE)
trainData, trainLabels, valData, valLabels = dataObj.loadTrainData(trainDataFile, trainLabelsFile)

trainDataMean = np.mean(trainData, axis=0)
trainData -= trainDataMean

valDataMean = np.mean(valData, axis=0)
valData -= valDataMean

print("Train Data", trainData.shape)
print("Train Labels", trainLabels.shape)

print("Validation Data", valData.shape)
print("Validation Labels", valLabels.shape)
print(valLabels[0], trainLabels[0])

model = Model(trainData[0].shape, NUM_CLASSES, LEARNING_RATE, EPOCHS)

lrScheduler = LearningRateScheduler(model.lr_schedule)
earlyStopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
modelCheckpoint = ModelCheckpoint(modelFile, monitor='val_loss', verbose=1, save_best_only=True)

model = model.model()
#model.load_weights(modelFile)

model.fit(
    trainData, trainLabels,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(valData, valLabels),
    epochs=EPOCHS,
    callbacks=[earlyStopping, lrScheduler, modelCheckpoint],
    verbose=1
)

Train Data (12000, 112, 112, 3)
Train Labels (12000, 10)
Validation Data (730, 112, 112, 3)
Validation Labels (730, 10)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Learning rate:  0.01
Train on 12000 samples, validate on 730 samples
Epoch 1/50
Learning rate:  0.001
12000/12000 [==============================] - 45s 4ms/step - loss: 1.8233 - accuracy: 0.3323 - val_loss: 1.9412 - val_accuracy: 0.3055

Epoch 00001: val_loss improved from inf to 1.94123, saving model to model64.h5
Epoch 2/50
Learning rate:  0.001
12000/12000 [==============================] - 39s 3ms/step - loss: 1.5176 - accuracy: 0.4571 - val_loss: 1.9509 - val_accuracy: 0.3479

Epoch 00002: val_loss did not improve from 1.94123
Epoch 3/50
Learning rate:  0.001
12000/12000 [==============================] - 39s 3ms/step - loss: 1.3293 - accuracy: 0.5375 - val_loss: 2.2562 - val_accuracy: 0.3986

Epoch 00003: val_loss did not improve from 1.94123
Epoch 4/50
Learning rate:  0.001
12000/12000 [==========

In [5]:
testDataFile = "/kaggle/input/minidata/test.npy" # sys.argv[1] 
testLabelsFile = "/kaggle/input/minidata/testlabels.npy" # sys.argv[2] 
modelFile = "model64.h5" # sys.argv[3]

BATCH_SIZE = 1 
NUM_CLASSES = 10

dataObj = Data(BATCH_SIZE) 
testData, testLabels = dataObj.loadTestData(testDataFile, testLabelsFile)

testDataMean = np.mean(testData, axis=0) 
testData -= testDataMean

print("Test Data", testData.shape) 
print("Test Labels", testLabels.shape)

model = load_model(modelFile) 
results = model.predict(testData)

count = 0 
for i, result in enumerate(results): 
    pred = np.argmax(result) 
    true = np.argmax(testLabels[i])
    # print(pred, true)
    if pred == true:
        count += 1
        
print("Accuracy- ", count/len(testData) * 100) 
print("Error- ", (1- (count/len(testData)))* 100)

Test Data (500, 112, 112, 3)
Test Labels (500, 10)
Accuracy-  73.4
Error-  26.6
